In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/2024 UW ENGINE Capstone/data/NERC_GIC/GICFolders

BASE_PATH = "/content/drive/MyDrive/2024 UW ENGINE Capstone/data/NERC_GIC/GICFolders"

/content/drive/.shortcut-targets-by-id/1oyb6sI_Gh8b_o0AIjqSCq4UljphkuJTI/2024 UW ENGINE Capstone/data/NERC_GIC/GICFolders


In [3]:
import pandas as pd
import os

In [4]:
dir_list = os.listdir(BASE_PATH)
print("Files and directories in :")
# prints all files
dir_list = dir_list[1:]
print(dir_list[1:])

Files and directories in :
['20150317.csv', '20150622.csv', '20150911.csv', '20150919.csv', '20151006.csv', '20151220.csv', '20170527.csv', '20170907.csv', '20170927.csv', '20180825.csv', '20210512.csv', '20211103.csv', '20220409.csv', '20230226.csv', '20230323.csv', '20230423.csv', 'stations_locations.csv']


In [5]:
df = pd.read_csv('20150317.csv')

In [6]:
df[['Lat', 'Long']].drop_duplicates()

,Lat,Long
0,41.7,87.5
325,41.8,88.0
650,40.2,75.3
1300,42.8,82.5
1625,46.6,87.4
...,...,...
34265,38.5,76.6
34915,39.5,75.1
35240,39.4,74.6
35890,31.4,95.4


In [7]:
distinct_locations = pd.DataFrame()

for file in dir_list[1:]:
    if file != "stations_locations.csv":
        print(file)
        df = pd.read_csv(file)
        df = df[['Lat', 'Long']].drop_duplicates()
        distinct_locations = pd.concat([distinct_locations, df], ignore_index=True)
        distinct_locations = distinct_locations[['Lat', 'Long']].drop_duplicates()

20150317.csv
20150622.csv
20150911.csv
20150919.csv
20151006.csv
20151220.csv
20170527.csv
20170907.csv
20170927.csv
20180825.csv
20210512.csv
20211103.csv
20220409.csv
20230226.csv
20230323.csv
20230423.csv


In [8]:
distinct_locations

,Lat,Long
0,41.7,87.5
1,41.8,88.0
2,40.2,75.3
3,42.8,82.5
4,46.6,87.4
...,...,...
288,27.9,97.4
289,32.8,94.5
290,42.5,87.9
291,41.6,86.3


In [9]:
distinct_locations.to_csv("devices_locations.csv")

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

In [ ]:
print(distinct_locations.Long.max())
print(distinct_locations.Long.min())

123.0
66.4


In [ ]:
print(distinct_locations.Lat.max())
print(distinct_locations.Lat.min())

48.0
26.1


In [ ]:
stations_locations = pd.read_csv("stations_locations.csv", index_col=[0])
stations_locations

,glon,glat
0,2.930,36.850
1,11.520,3.870
2,39.460,14.280
3,39.460,14.280
4,355.870,5.330
...,...,...
555,37.730,54.940
556,12.250,-83.340
557,311.487,-1.205
558,15.520,47.550


In [ ]:
print(stations_locations.glon.max())
print(stations_locations.glon.min())


358.82
0.493


In [ ]:
stations_locations['glon_new'] = -stations_locations['glon']
distinct_locations.Long = -distinct_locations['Long'] + 360

In [ ]:

# Written by Sean Jung
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# print(distinct_locations)
# Define the radius of the circle in miles (adjust as needed)

# Plotting using Plotly
fig = px.scatter_geo(distinct_locations, lat='Lat', lon='Long')
# fig.add_trace()
fig.add_scattergeo(lat=stations_locations['glat'], lon=stations_locations['glon'], mode='markers', name='Stations')
fig.update_layout(mapbox={"style": "carto-positron"})

for station in stations_locations.iterrows():
    fig.add_trace(
        go.Scattergeo(
            lon=[station[1]['glon']],
            lat=[station[1]['glat']],
            mode='markers',
            marker=dict(size=15, color='red', opacity=0.1),
            showlegend=False,
        )
    )
fig.show()

In [ ]:
import geopy.distance

coords_1 = (41.7, 87.5)
coords_2 = (42.0, 87.9)

print(geopy.distance.geodesic(coords_1, coords_2).miles)

29.235648549349854


In [ ]:
from math import radians, sin, cos, sqrt, atan2

def distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth given their latitude and longitude
    using the Haversine formula.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula: https://en.wikipedia.org/wiki/Haversine_formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    earth_radius_miles = 3958.8  # Radius of the Earth in miles
    distance = earth_radius_miles * c  # Convert distance to miles
    return distance

In [ ]:
distance(41.7, 87.5, 42.0, 87.9)

29.21445823356088